### Import the libraries

In [ ]:
#*Import the libraries*
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
import plotly.figure_factory as ff
import plotly.graph_objects as go



# ELT
### Import the data

'USER_ID'
'PRODUCT_ID'
'RATINGS'
'TIMESTAMP'

In [ ]:
#define column name of the csv
cols_name=['USER_ID', 'PRODUCT_ID', 'RATINGS','TIMESTAMP']
# import the csv
amz_kindle_store=pd.read_csv('Kindle_Store.csv',names=cols_name)
# run the follow code to see the df
amz_kindle_store.head()

In [ ]:
amz_kindle_store.info()


Check the number of rows and columns

In [ ]:
rows,columns=amz_kindle_store.shape
print('The number of rows is: ',rows)
print('Number of columns is: ',columns)

# EDA

### Count of NaNs

there isn't null values

In [ ]:
counts = amz_kindle_store.isna().sum()
print(counts.sort_values())


percentages = round(amz_kindle_store.isna().mean() * 100, 1)
print(percentages.sort_values())

### Some summary indicators

In [ ]:
#Summary statistics of rating variable
round(amz_kindle_store['RATINGS'].describe().transpose(),2)

In [ ]:
fig = go.Figure(data=[go.Histogram(x=amz_kindle_store['RATINGS'],
                                   texttemplate="%{x}", 
                                   textfont_size=12)])
fig.show()

#### How many user and product?

In [ ]:
# Number of unique user id  in the data
print('Number of distinct users is = ', amz_kindle_store['USER_ID'].nunique())
# Number of unique product id  in the data
print('Number of distinct product is = ', amz_kindle_store['PRODUCT_ID'].nunique())

In [ ]:
#Check the top 5 users/product based on ratings
userid_most_rated=amz_kindle_store.groupby('USER_ID').size().sort_values(ascending=False)[:5]
productid_most_rated=amz_kindle_store.groupby('PRODUCT_ID').size().sort_values(ascending=False)[:5]
print('Top 5 users based on ratings: \n',userid_most_rated)
print('Top 5 product based on ratings: \n',productid_most_rated)

In [ ]:
count_user_amz_kindle_store.columns



# Model Preparation

Split the data randomnly into train and test datasets into 70:30 ratio




In [ ]:
amz_kindle_train_data, amz_kindle_test_data = train_test_split(amz_kindle_store, test_size = 0.3, random_state=0)


In [ ]:
print('Shape of training data: ',amz_kindle_train_data.shape)
print('Shape of testing data: ',amz_kindle_store.shape)

# Model 1: Building Popularity Recommder model

The Popularity-based recommender system is a non-personalised recommender system and these are based on frequecy counts, which may be not suitable to the user.We can see the differance above for the user id 4, 6 & 8, The Popularity based model has recommended the same set of 5 products to both but Collaborative Filtering based model has recommended entire different list based on the user past purchase history.
hese systems check about the product or movie which are in trend or are most popular among the users and directly recommend those.